In [ ]:
import numpy as np
import argopy
from argopy import DataFetcher
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import gsw
from datetime import datetime
from cartopy.geodesic import Geodesic
from scipy import linalg
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as C
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import haversine_distances
from matplotlib.patches import Circle

import ArgoHeatContentDataCollater
import ArgoGPR

In [ ]:
# 1. LOAD DATA
# Adjust path to your folder
df_argo = load_argo_data_advanced(
    nc_dir='./home/avik2007/ArgoEBUSAnalysis/', 
    start_date='2018-01-01', 
    end_date='2018-03-01', # Grab a few months to test
    lat_bounds=[30, 40], 
    lon_bounds=[-126, -119]
)

# 2. RUN LOOO (Baseline Check)
# This validates if the model works mathematically
z_looo = generalized_cross_validation(
    df_argo, 
    feature_cols=['lat', 'lon'], # 2D Spatial Only for now
    method='LOOO'
)

# 3. RUN LOFO (The "Hard" Test)
# This validates if the model works SCIENTIFICALLY
z_lofo = generalized_cross_validation(
    df_argo, 
    feature_cols=['lat', 'lon'], 
    method='LOFO'
)